### Вопросы

- Пайплайн создания модели в реальной жизни,откуда берем данные(Hdoop,БД итд)
- [Преобразователь данных которые идут на вход во время деплоя ,как это все хранится(классы,функции) best practice](#1).
    - Использовать встроенные Imputer ,если использовать самописные то как его потом применить
    - Чистота кода
- [Хранение различных артефактов](#2).
- [Используется ли вообще декодирование категориальных признаков и декодирование в числа после нормолизации](#3).
- [Пайплайн подбора модели ,нужно ли автоматизировать этот процесс в реальной жизни](#4).
- Частота переобучения модели ,подходы к конвейеру обучения/переобучения


<a id='0'></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import math
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error,mean_absolute_error

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso,Ridge

#Список моделей которые будем использовать

#Модели регрессии
#Gradient Boosting Regression
from sklearn.ensemble import GradientBoostingRegressor
#Elastic Net Regression
from sklearn.linear_model import ElasticNet
#Stochastic Gradient Descent Regression
from sklearn.linear_model import SGDRegressor
#Support Vector Machine
from sklearn.svm import SVR
#Bayesian Ridge Regression
from sklearn.linear_model import BayesianRidge
#CatBoost Regressor
from catboost import CatBoostRegressor
#Kernel Ridge Regression
from sklearn.kernel_ridge import KernelRidge
#Linear Regression
from sklearn.linear_model import LinearRegression
#Ridge and Lasso regression
#XGBoost Regressor
from xgboost import XGBRegressor
#LGBM Regressor
#from lightgbm import LGBMRegressor

# Подбор гиперпараметров

import optuna

%matplotlib inline


In [ ]:
#Ставим кол-во выводимых строк и столбцов 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

#Отключаем предупреждения

warnings.filterwarnings('ignore')

### Data Loading

In [ ]:
#Загружаем данные

train_data = 'train.csv'
test_data = 'test.csv'

with open(train_data) as train:
    train_df = pd.read_csv(train)
    
with open(test_data) as test:
    test_df = pd.read_csv(test)

In [ ]:
train_df_copy = train_df.copy(deep=True)

In [ ]:
train_df_copy.shape

In [ ]:
train_df_copy.describe()

In [ ]:
#Проверка на дубли

train_df_copy[train_df_copy.duplicated()]


In [ ]:
#Удаляем дубли

train_df_copy.drop_duplicates(inplace = True)

In [ ]:
#Описание данных 

train_df_copy.describe().loc[['min', 'max', 'mean', 'std']].round(2)

### EDA

In [ ]:
#Функции для создания таблицы по пропускам и визуализации фичей с пропусками

def missings(data_frame,missings_percent):

    total = data_frame.isnull().sum().sort_values(ascending=False)
    percent = (data_frame.isnull().sum()/train_df.isnull().count()*100).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    missing_data = missing_data[(missing_data['Percent']>missings_percent)]
    
    return missing_data
    
def missings_bar_vizualisation(data_frame,missings_percent):
    
    try:

        total = data_frame.isnull().sum().sort_values(ascending=False)
        percent = (data_frame.isnull().sum()/train_df.isnull().count()*100).sort_values(ascending=False)
        missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        missing_data = missing_data[(missing_data['Percent']>missings_percent)]
    
        #plt.rcParams["figure.figsize"] = (10,5)
        sns.barplot(x=missing_data.index,y=missing_data['Percent'],data = missing_data)
        plt.xticks(rotation=70)
        plt.show()
    
    except ValueError:
        
        print('No missings')

In [ ]:
missings(train_df_copy,0)

In [ ]:
missings_bar_vizualisation(train_df_copy,0)

In [ ]:
#Дропним у кого больше percent пропусков

percent = 60

#Записываем таблицу с % пропусков

missings_table = missings(train_df_copy,percent)

#Получаем список колонок на удаление

for_drop = missings_table.index.to_list()

#Удаляем колонки
after_drops = train_df_copy.drop(columns = for_drop)


In [ ]:
for_drop

In [ ]:
#Проверяем что удалились колонки с процентом пропусков больше чем percent

missings_bar_vizualisation(after_drops,percent)

In [ ]:
after_drops.shape

In [ ]:
#Смотрим на оставшуюся таблицу

after_drops.head(5)

In [ ]:
#Дропним поле Id за ненадобностью

after_drops.drop(columns = 'Id',inplace=True)

In [ ]:
#Список полей у которых есть пропуски

list_of_missing_columns = missings(after_drops,0).index.to_list()


In [ ]:
list_of_missing_columns

In [ ]:
missings_bar_vizualisation(after_drops,0)

In [ ]:
after_drops.dtypes

#### Преобразователь данных которые идут на вход во время деплоя ,как это все хранится(классы,функции) best practice
<a id='1'></a>

[Вопросы](#0)


In [ ]:
#Заполним пропуски по всем переменным. Для числовых ставим median() для категориальных "None"
#Стратегий заполнения пропусков может быть много более продвинутая это заполнение относительно целевой переменной

#В реальных данных пропущенными могут быть абсолютно любые поля поэтому заполнение производим по всем полям которые есть в датафрейме 
#после удаления ненужных колонок даже если там нет пропусков

#Float и int можно объединить в один if

unique_types = after_drops.dtypes.unique()

#Для простоты заполним медианным значением

for typ in unique_types:
    
    if typ == 'object':
        
        object_columns = after_drops.select_dtypes(include = ['object']).columns.to_list()
        after_drops[object_columns] = after_drops[object_columns].fillna('None')
    
    elif typ == 'float64':
        
        float_columns = after_drops.select_dtypes(include = ['float64']).columns.to_list()
        after_drops[float_columns] = after_drops[float_columns].fillna(after_drops[float_columns].median())

    
    elif typ == 'int64':
        
        int_columns = after_drops.select_dtypes(include = ['int64']).columns.to_list()
        after_drops[int_columns] = after_drops[int_columns].fillna(after_drops[int_columns].median())

In [ ]:
#Артефакты по заполнению пропусков для числовых переменных

after_drops[float_columns].median()

In [ ]:
after_drops[int_columns].median()

In [ ]:
int_float_artefacts = pd.concat([after_drops[float_columns].median(),after_drops[int_columns].median()])

In [ ]:
int_float_artefacts

In [ ]:
#Еще раз проверяем что все заполнилось 

missings_bar_vizualisation(after_drops,0)

In [ ]:
# Функция для визуализации hist plot ,boxplot,countplot и Scatter plot для всех признаков

def overall_viz(data_frame,cols,cell):

    heart_CAT = data_frame.columns.to_list()

#  Categorical Data

    b = cols  # number of columns
    a = math.ceil(len(heart_CAT)/b)*3  # number of rows
    c = 1  # initialize plot counter

    fig = plt.figure(figsize = (20,200))



    for i in heart_CAT:
        
        if data_frame[i].dtype in ['float64','int64']:
            
            plt.subplot(a, b, c)
            plt.xlabel(i)
            sns.boxplot(data = data_frame, x = i)
            c = c + 1
    
            plt.subplot(a, b, c)
            plt.xlabel(i)
            sns.histplot(data = data_frame, x = i)
            c = c + 1
            
            plt.subplot(a, b, c)
            plt.xlabel(i)
            sns.scatterplot(data=data_frame, x=cell, y=i)
            c = c + 1
        
        elif data_frame[i].dtype in ['object']:
            
            plt.subplot(a, b, c)
            plt.xlabel(i)
            sns.countplot(data = data_frame, x = i)
            c = c + 1
            
            plt.subplot(a, b, c)
            plt.xlabel(i)
            sns.histplot(data = data_frame, x = i)
            c = c + 1
            
            plt.subplot(a, b, c)
            plt.xlabel(i)
            sns.scatterplot(data=data_frame, x=cell, y=i)
            c = c + 1
        
        
#QQ Plot    
#    plt.subplot(a, b, c)
#    plt.xlabel(i)
#    pplot(data = after_drops_numeric,x = i,kind='qq')
#    c = c + 1

#fig.tight_layout(h_pad=2,w_pad=2)
    
    plt.show()

In [ ]:
overall_viz(after_drops,3,"SalePrice")

In [ ]:
#Сохраним список категориальных признаков

categorical_features = after_drops.select_dtypes(include=['object']).columns.to_list()

In [ ]:
categorical_features

In [1]:
#Преобразование категориальных признаков

#Lable encoding


cols = after_drops.select_dtypes(include = ['object']).columns.to_list()

# process columns, apply LabelEncoder to categorical features

transform_df = pd.DataFrame()

for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(after_drops[c].values)) 
    after_drops[c] = lbl.transform(list(after_drops[c].values))

#Создадим словарь для последующего декодирования
    
    keys = lbl.classes_
    values = lbl.transform(lbl.classes_)
    transform = pd.DataFrame(zip(keys, values),columns = ['name','transform_value'])
    transform['feature_name'] = c
    transform_df = pd.concat([transform,transform_df])

# shape        
#print('Shape all_data: {}'.format(after_drops.shape),)



SyntaxError: invalid syntax (3276747912.py, line 28)

#### Используется ли вообще декодирование категориальных признаков и декодирование в числа после нормолизации

<a id='3'></a>

[Вопросы](#0)

In [ ]:
#Можно увидеть, что None декодируется например 4, хотя возможно что 0 было бы более подходящее значение
#Артефакт

transform_df

In [ ]:
#Проверим что все преобразовалось

after_drops.select_dtypes(include = ['object']).head(2)

In [ ]:
after_drops.head(5)

In [ ]:
overall_viz(after_drops,3,"SalePrice")

### К текущему этапу наш датафрейм очищен от больших пропусков и преобразованны категориальные переменные
#### Преобразование категориальных переменных можно было бы и опустить если мы будем использовать catboost, которые принимает на вход категориальные переменные.
#### Так же можно было бы детальнее поработать с заполнением пропусков для категориальных переменных а именно заполнить их не None а более умным способом.
#### Обратное декодирование было сделано с целью тренеровки оно так же скорее всего не понадобится в дальнейшем

#### Хранение различных артефактов

<a id='2'></a>

[Вопросы](#0)

In [ ]:
#Корреляция относительно целевой переменной

plt.figure(figsize=(14,8))
after_drops.corr()['SalePrice'].sort_values(ascending=False).plot(kind='bar')

### Model train an selection

<a id='4'></a>

[Вопросы](#0)

In [ ]:
#Разделим набор данных и целевую переменную

X_train = after_drops.loc[:,:'SaleCondition']
Y_train = after_drops['SalePrice']

#Нормализация данных

#В дальнейшем мы не будем использовать нормализацию но пусть будет тут
scaler = MinMaxScaler().fit(X_train)
x_minmax = scaler.transform(X_train)


In [ ]:
# Возьмем сразу несколько моделей и сделаем поиск по сетке
#Параметры по моделям Регрессоров(Надо засунуть в yaml)

catboost_params = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 6,10]
         }

lasso_params = {
    'alpha':[0.01,0.1,0,10,100,200,300,500]
        }

ridge_params = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}

xgb_params ={"learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4]
            }

DecisionTreeRegressor_params = [
    {'max_depth': [3, 5, 10], 
     'max_features': [3, 4, 5]},
    {'random_state': [0, 1, 2, 3, 4], 
     'min_samples_split': [2, 3, 4]}    
]

ElasticNet_params = [
        {'alpha': [.0004, .0005], 'l1_ratio': [.5, .8, 1]}
]

#Словарь модель:параметры

models_dict = {'models':
               [CatBoostRegressor(),
                Lasso(),
                XGBRegressor(),
                Ridge(),
                DecisionTreeRegressor(),
                ElasticNet()
               ],
               
               'params':[catboost_params,
                         lasso_params,
                         xgb_params,
                         ridge_params,
                         DecisionTreeRegressor_params,
                         ElasticNet_params
                        ]
              }

error = 'r2'


In [ ]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

####Блок по моделям

def models_cyckle(X_train,Y_train,folds,error):

    models_report = pd.DataFrame()
    
    for i in range(len(models_dict['models'])):
        
        
          
        grid_search = GridSearchCV(models_dict['models'][i], param_grid=models_dict['params'][i], cv=folds, scoring=error, n_jobs=-1)
        grid_search.fit(X_train, Y_train)
    
        best_model = pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score').head(1)
        best_model['model_name'] = str(models_dict['models'][i])
        models_report = pd.concat([models_report,best_model])
    
    return models_report

In [ ]:
report = models_cyckle(X_train,Y_train,folds,error)

In [ ]:
report.loc[:,:'model_name'].sort_values(by='mean_test_score',ascending = False)

In [ ]:
#Получаем что лучше всего с задачей справился XGBRegressor

In [ ]:
#Теперь самое интересное.
#Надо взять наш тестовый датасет и произвести над ним все манипуляции, которые производили с train.
#Заполнить пропуски,закодировать категориальные переменные

#### Артефакты к данному моменту, которые нужно будет потом преобразовать в python код
##### Список полей которые мы использовали при обучении(мы не отбрасывали ни какие признаки на основании аналитического исследования, а отбрасывали только пустые)
##### Заполнение пропусков
##### Кодирование категориальных переменных

In [ ]:
#Выбираем XGBRegressor с такими параметрами

#{'colsample_bytree': 0.3, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 1}

xgb_params ={"learning_rate": 0.1,
                        "max_depth": 4,
                        "min_child_weight":1,
                        "gamma":0.1,
                        "colsample_bytree":0.3
            }


In [ ]:
#Передаем в модель наши параметры

model = XGBRegressor(**xgb_params).fit(X_train, Y_train)


In [ ]:
# Сохраним нашу модель
import pickle

with open("XGBRegressor.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
# Пишем класс для преобразования пропусков которы потом преобразуем в python файл

передаем значения или датафрейм в функцию
оставляем только нужные колонки которые использовали при обучении

In [ ]:
used_columns = after_drops.loc[:,:'SaleCondition'].columns.to_list()

In [ ]:
test_df[used_columns].head(5)

In [ ]:
def Imputer(dataframe,used_columns):

    #Оставляем поля которые использовали при обучении
    
    used_df = dataframe[used_columns]

    #Заменяем пропуски с помощью артефактов полученных на тренеровочном датасете

    unique_types = used_df.dtypes.unique()

#Для простоты заполним медианным значением

    for typ in unique_types:
    
        if typ == 'object':
        
            object_columns = used_df.select_dtypes(include = ['object']).columns.to_list()
            used_df[object_columns] = used_df[object_columns].fillna('None')

        elif typ != 'object':
            
            non_object_columns = used_df.select_dtypes(exclude = ['object']).columns.to_list()
            
            for i in non_object_columns:
                
                used_df[i] = used_df[i].map(transform_df[['name','transform_value']][transform_df['feature_name'] == i].set_index('name').to_dict()['transform_value'])
            
            used_df.map(int_float_artefacts)
    
        

    return used_df

In [ ]:
int_float_artefacts.index

In [ ]:
int_float_artefacts

In [ ]:
Imputer(train_df,used_columns)

In [ ]:
#Артефакты по заполнению пропусков медианным значением
int_float_artefacts

In [ ]:
#Преобразуем пропуски



In [ ]:
unique_types = after_drops.dtypes.unique()

#Для простоты заполним медианным значением

for typ in unique_types:
    
    if typ == 'object':
        
        object_columns = after_drops.select_dtypes(include = ['object']).columns.to_list()
        after_drops[object_columns] = after_drops[object_columns].fillna('None')
    
    elif typ == 'float64':
        
        float_columns = after_drops.select_dtypes(include = ['float64']).columns.to_list()
        after_drops[float_columns] = after_drops[float_columns].fillna(after_drops[float_columns].median())

    
    elif typ == 'int64':
        
        int_columns = after_drops.select_dtypes(include = ['int64']).columns.to_list()
        after_drops[int_columns] = after_drops[int_columns].fillna(after_drops[int_columns].median())

In [ ]:
#Берем только колонки на которых обучались

test_df_2 = test_df[X_train.columns]

In [ ]:
#Наши артефакты

#Перевод категориальных переменных в числовые из LabelEncoder
transform_df 

#Заполнение пропусков в числовых переменных

after_drops[float_columns].median()

In [ ]:
#Список полей у которых есть пропуски

list_of_missing_columns_2 = missings(test_df_2,0).index.to_list()

In [ ]:
list_of_missing_columns_2

In [ ]:
#Заполним пропуски по всем переменным. Для числовых ставим median() для категориальных "None"

unique_types = test_df_2[list_of_missing_columns_2].dtypes.unique()


for typ in unique_types:
    
    if typ == 'object':
        
        object_columns = test_df_2[list_of_missing_columns_2].select_dtypes(include = ['object']).columns.to_list()
        test_df_2[object_columns] = test_df_2[object_columns].fillna('None')
    
    elif typ == 'float64':
        
        float_columns = test_df_2[list_of_missing_columns_2].select_dtypes(include = ['float64']).columns.to_list()
        test_df_2[float_columns] = test_df_2[float_columns].fillna(after_drops[float_columns].median())

    
    elif typ == 'int64':
        
        int_columns = test_df_2[list_of_missing_columns_2].select_dtypes(include = ['int64']).columns.to_list()
        test_df_2[int_columns] = test_df_2[int_columns].fillna(after_drops[int_columns].median())

In [ ]:
#Еще раз проверяем что все заполнилось 

missings_bar_vizualisation(test_df_2,0)

In [ ]:
test_df_2.head(5)

In [ ]:
#Декодирование категориальных переменных
transform_df

In [ ]:
#Само декодирование

obj_col = test_df_2.select_dtypes('object').columns.to_list()
for i in obj_col:
    test_df_2[i] = test_df_2[i].map(transform_df[['name','transform_value']][transform_df['feature_name'] == i].set_index('name').to_dict()['transform_value'])



In [ ]:
test_df_2.shape

In [ ]:
test_df_2[obj_col]

In [ ]:
after_drops.shape

In [ ]:
#Делаем предсказание

prediction = model.predict(test_df_2)

In [ ]:
prediction

In [ ]:
#Submission

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_df['Id']
sub['SalePrice'] = prediction
sub.to_csv('submission.csv',index=False)

In [ ]:
sub

#### Выводы:

- 1.Не поработал с аномалиями
- 2.Сложная система декодирования - но я хотел попробовать ее сделать.
- 3.Нету ФичаИнжениринга
- 4.Надо упростить код - убрать функции всякие чтоб стало прозрачнее
- 5.Заполнение пропусков - можно использовать что то более интересное KNNImputer например
- 6.Декодирование - можно посмотреть другие подходы
- 7.Параметры моделей можно убрать в Yaml или ini файл
- 8.Для большого числа экспериментов когда хочется попробовать разные импутеры и кодировщики надо использовать какой-то автоматизированный подход Airflow,MLFlow,Hydra,Piplines
- 9.Вообще не читал описание данных.
- 10.Разобраться с метриками
- 11.Для CatBoost не обязательно делать кодирование
- 12.Не использовал нормализацию
- 13.Не проверил как прошло декодирование
- 14.Попробовать AutoML
- 15.Попробовать подбор параметров модели через Optuna
- 16.Использовать готовые фреймворки для построения графиков
- 17.PSI для проверки стабильности фич во времени


- 15.Попробовать подбор параметров модели через Optuna

In [ ]:
study = optuna.create_study(study_name="my_first_study", direction="maximize")

In [ ]:
xgb_params ={"learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4]
            }

In [ ]:
ridge_params = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}

In [ ]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

def objective(trial):
    # предложение гиперпараметров
    alpha = trial.suggest_float('alpha', 1e-10, 1.0, log=True)

    # Модель
    model = Ridge(alpha=alpha)

    # предсказание и вычисление MSE
    model.fit(X_train, Y_train)
    error = cross_val_score(model, X_train, Y_train, n_jobs=-1, cv=folds,scoring = 'r2')
    #mse = mean_squared_error(y_valid, y_pred)
    
    return error

study = optuna.create_study(directions=['maximize'])
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("Error":)
print("  Values: ", trial.values)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
models_dict = {'models':
               [
                Ridge()
               ],
               
               'params':[
                         ridge_params
                         
                        ]
              }

error = 'r2'
def models_cyckle(X_train,Y_train,folds,error):

    models_report = pd.DataFrame()
    
    for i in range(len(models_dict['models'])):
        
        
          
        grid_search = GridSearchCV(models_dict['models'][i], param_grid=models_dict['params'][i], cv=folds, scoring=error, n_jobs=-1)
        grid_search.fit(X_train, Y_train)
    
        best_model = pd.DataFrame(grid_search.cv_results_).sort_values(by='rank_test_score').head(1)
        best_model['model_name'] = str(models_dict['models'][i])
        models_report = pd.concat([models_report,best_model])
    
    return models_report

In [ ]:
alpha = trial.suggest_float('alpha', 1e-10, 1.0, log=True)


In [ ]:
alpha